The evolution of a trajectory $ \mathbf{x}(t;t0,\mathbf{x_0}) $ as a function of its initial position $ \mathbf{x_0} $ is described by the flow map

$$ \mathbf{F}_{t_0}^t : \mathbf{x}_0 \rightarrow \mathbf{x}(t;t_0,\mathbf{x_0}) $$

Differentiation of the flow map $ {\mathbf{F}_1}_{t_0}^t = \begin{pmatrix} {F_1}_{t_0}^t \\ {F_2}_{t_0}^t \end{pmatrix} $ with respect to time gives:

\begin{equation}
\dfrac{d}{dt}\mathbf{F}_{t_0}^t(\mathbf{x_0}) = \mathbf{v}(\mathbf{F}_{t_0}^t(\mathbf{x_0}), t) = \begin{pmatrix} u((\mathbf{F}_{t_0}^t(\mathbf{x_0}), t)) \\ v((\mathbf{F}_{t_0}^t(\mathbf{x_0}), t)) \end{pmatrix}
\label{eq: dFdt},
\end{equation}

with $ \mathbf{v}(\mathbf{F}_{t_0}^t(\mathbf{x_0}), t) $ being the velocity field.

Taking the gradient of \ref{eq: dFdt}, it holds that $ D \mathbf{F}_{t_0}^t(\mathbf{x_0}) $ evolves according to the equations of variations:

\begin{equation}
\dfrac{d}{dt} D \mathbf{F}_{t_0}^t(\mathbf{x_0}) = D\mathbf{v}(\mathbf{F}_{t_0}^t(\mathbf{x_0}), t) D \mathbf{F}_{t_0}^t(\mathbf{x_0}) = \begin{pmatrix} \partial_x u && \partial_y u \\ \partial_x v && \partial_y v \end{pmatrix} \begin{pmatrix} D_x{F_1}_{t_0}^t && D_y{F_1}_{t_0}^t \\ D_x{F_2}_{t_0}^t && D_y{F_2}_{t_0}^t \end{pmatrix}
\label{eq: dDFdt}
\end{equation}

Equations \ref{eq: dFdt} and \ref{eq: dDFdt} lead to a system of coupled ODEs.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| t | float | time |
| x | array (3,) | $ \mathbf{x} $ |
| X | array (Ny, Nx, Nz) | X-meshgrid|
| Y | array (Ny, Nx, Nz) | Y-meshgrid|
| Z | array (Ny, Nx, Nz) | Z-meshgrid|
| Interpolant | list (3,) | Interpolant[0]: Interpolant object for $ u(\mathbf{x}, t)  $ <br /> Interpolant[1]: Interpolant object for $ v(\mathbf{x}, t)  $ <br /> Interpolant[2]: Interpolant object for $ w(\mathbf{x}, t)  $ |
| periodic | list (3,) | periodic[0]: periodicity in x <br /> periodic[1]: periodicity in y <br /> periodic[2]: periodicity in z <br />|
| bool_unsteady | bool | specifies if velocity field is unsteady/steady |
| v | float | $ u(\mathbf{x}, t) $ |
| v | float | $ v(\mathbf{x}, t) $ |
| w | float | $ w(\mathbf{x}, t) $ |
| DxF1 | float | $ D_x{F_1}_{t_0}^t(mathbf{x_0}) $ |
| DyF1 | float | $ D_y{F_1}_{t_0}^t(mathbf{x_0}) $ |
| DxF2 | float | $ D_x{F_2}_{t_0}^t(mathbf{x_0}) $ |
| DyF2 | float | $ D_y{F_2}_{t_0}^t(mathbf{x_0}) $ |

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-1])

# add Algorithm folder to current working path in order to access the functions
sys.path.append(parent_directory+"/utils")

In [2]:
# Import numpy
import numpy as np

# Import function that computes velocity
from ipynb.fs.defs.velocity import velocity

# Import function that computes gradient of velocity
from ipynb.fs.defs.gradient_velocity import gradient_velocity

In [3]:
def dFdt_eqVar(t, x, X, Y, Interpolant, periodic, defined_domain, bool_unsteady, aux_grid):
    
    # Flow map F at time t
    F = x[:2]
    
    # first row of DF
    DxF1, DyF1, DyF1 = x[2], x[3], x[4]
    
    # second row of DF
    DxF2, DyF2, DyF2 = x[5], x[6], x[7]
    
    # third row of DF
    DxF3, DyF3, DyF3 = x[8], x[9], x[10]
    
    # check if periodic in x
    if periodic[0]:
        
        x[0] = x[0]%(X[0, -1, 0]-X[0, 0, 0])
    
    # check if periodic in y
    if periodic[1]:
        
        x[1] = x[1]%(Y[-1, 0, 0]-Y[0, 0, 0])
    
    # check if periodic in y
    if periodic[2]:
        
        x[2] = x[2]%(Z[0, 0, -1]-Z[0, 0, 0])
    
    # compute velocity
    vel = velocity(t, F, X, Y, Z, Interpolant, periodic, defined_domain, bool_unsteady)
    
    # compute gradient of velocity
    grad_vel = gradient_velocity(t, F, X, Y, Interpolant, periodic, defined_domain, bool_unsteady, aux_grid)
    
    # eq. 2
    dDFdt = grad_vel@DF
    
    # eq.1
    u = vel[0]
    v = vel[1]
    w = vel[2]
    
    # extract elements of dDFdt
    dDxF1dt = dDFdt[0,0]
    dDyF1dt = dDFdt[0,1]
    dDzF1dt = dDFdt[0,2]
    dDxF2dt = dDFdt[1,0]
    dDyF2dt = dDFdt[1,1]
    dDzF2dt = dDFdt[1,2]
    dDxF3dt = dDFdt[2,0]
    dDyF3dt = dDFdt[2,1]
    dDzF3dt = dDFdt[2,2]
    
    return [u, v, w, dDxF1dt, dDyF1dt, dDzF1dt, dDxF2dt, dDyF2dt, dDzF2dt, dDxF3dt, dDyF3dt, dDzF3dt]
    